In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, TimeDistributed, Flatten, BatchNormalization
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = 'data/basic.csv'
data = pd.read_csv(file_path)

# Remove unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Handle missing values (e.g., replace with 0)
data = data.fillna(0)

# Extract player_id that exists in all years 2016, 2017, 2018, 2019
data_2016 = data[data['year'] == 2016]
data_2017 = data[data['year'] == 2017]
data_2018 = data[data['year'] == 2018]
data_2019 = data[data['year'] == 2019]

player_ids_2016 = set(data_2016['player_id'].unique())
player_ids_2017 = set(data_2017['player_id'].unique())
player_ids_2018 = set(data_2018['player_id'].unique())
player_ids_2019 = set(data_2019['player_id'].unique())

common_player_ids = player_ids_2016 & player_ids_2017 & player_ids_2018 & player_ids_2019

# Extract data corresponding to common player_id
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data corresponding to the years 2016, 2017, 2018
final = common_data[common_data['year'].isin([2016, 2017, 2018])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 3  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data to be the same as the entire data
X_train, y_train = X_seq, y_seq

In [2]:
# LSTM_A

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_A
    model_A = Sequential()
    model_A.add(LSTM(128, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_A.add(LSTM(128, return_sequences=True))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Flatten())
    model_A.add(Dense(1024, activation="relu"))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_A.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_A = model_A.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter data for the year 2019
    data_19 = common_data[common_data['year'] == 2019]

    # Scale the data for the year 2019
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Convert to time series data format (using data from 2016, 2017, 2018 to predict 2019)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict data for the year 2019
    y_pred_scaled_A = model_A.predict(X_2019_seq)

    # Inverse scale the predictions
    y_pred_A = scaler_y.inverse_transform(y_pred_scaled_A)

    # Restore the actual 2019 p_era values for comparison
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse_A = np.sqrt(mean_squared_error(y_test_actual, y_pred_A))
    rmse_list.append(rmse_A)

    # Calculate MAE
    mae_A = mean_absolute_error(y_test_actual, y_pred_A)
    mae_list.append(mae_A)
    
    # Calculate MAPE
    mape_A = mean_absolute_percentage_error(y_test_actual, y_pred_A)
    mape_list.append(mape_A)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_A}, MAE: {mae_A}, MAPE: {mape_A}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 17s 531ms/step - loss: 0.0760 - accuracy: 0.0000e+00 - val_loss: 0.0393 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 34ms/step - loss: 0.0317 - accuracy: 0.0025 - val_loss: 0.0292 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 38ms/step - loss: 0.0254 - accuracy: 0.0025 - val_loss: 0.0342 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 33ms/step - loss: 0.0228 - accuracy: 0.0025 - val_loss: 0.0286 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 37ms/step - loss: 0.0210 - accuracy: 0.0025 - val_loss: 0.0292 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 35ms/step - loss: 0.0197 - accuracy: 0.0025 - val_loss: 0.0279 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 36ms/step - loss: 0.0182 - accuracy: 0.0025 - val_loss: 0.0266 - val_accurac

In [3]:
# LSTM_B

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_B
    model_B = Sequential()
    model_B.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_B.add(LSTM(16, return_sequences=True))
    model_B.add(LSTM(8, return_sequences=True))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Flatten())
    model_B.add(Dense(512, activation="relu"))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Dense(64, activation="relu"))
    model_B.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_B.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_B = model_B.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter data for the year 2019
    data_19 = common_data[common_data['year'] == 2019]

    # Scale the data for the year 2019
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Convert to time series data format (using data from 2016, 2017, 2018 to predict 2019)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict data for the year 2019
    y_pred_scaled_B = model_B.predict(X_2019_seq)

    # Inverse scale the predictions
    y_pred_B = scaler_y.inverse_transform(y_pred_scaled_B)

    # Restore the actual 2019 p_era values for comparison
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse_B = np.sqrt(mean_squared_error(y_test_actual, y_pred_B))
    rmse_list.append(rmse_B)

    # Calculate MAE
    mae_B = mean_absolute_error(y_test_actual, y_pred_B)
    mae_list.append(mae_B)
    
    # Calculate MAPE
    mape_B = mean_absolute_percentage_error(y_test_actual, y_pred_B)
    mape_list.append(mape_B)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_B}, MAE: {mae_B}, MAPE: {mape_B}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 20s 609ms/step - loss: 0.1226 - accuracy: 0.0025 - val_loss: 0.0427 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 28ms/step - loss: 0.0374 - accuracy: 0.0025 - val_loss: 0.0290 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 25ms/step - loss: 0.0243 - accuracy: 0.0025 - val_loss: 0.0454 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0267 - accuracy: 0.0025 - val_loss: 0.0341 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 24ms/step - loss: 0.0230 - accuracy: 0.0025 - val_loss: 0.0282 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 25ms/step - loss: 0.0215 - accuracy: 0.0025 - val_loss: 0.0319 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 33ms/step - loss: 0.0214 - accuracy: 0.0025 - val_loss: 0.0299 - val_accuracy: 0

In [5]:
# LSTM_C

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_C
    model_C = Sequential()
    model_C.add(Masking(mask_value=0.0, input_shape=(seq_length, X_train.shape[2])))
    model_C.add(LSTM(64, return_sequences=True))
    model_C.add(LSTM(64, return_sequences=True))
    model_C.add(Dropout(rate=0.5))
    model_C.add(TimeDistributed(Dense(10)))
    model_C.add(Flatten())
    model_C.add(Dense(512, activation="relu"))
    model_C.add(Dropout(rate=0.5))
    model_C.add(Dense(64, activation="relu"))
    model_C.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_C.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_C = model_C.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter data for the year 2019
    data_19 = common_data[common_data['year'] == 2019]

    # Scale the data for the year 2019
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Convert to time series data format (using data from 2016, 2017, 2018 to predict 2019)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict data for the year 2019
    y_pred_scaled_C = model_C.predict(X_2019_seq)

    # Inverse scale the predictions
    y_pred_C = scaler_y.inverse_transform(y_pred_scaled_C)

    # Restore the actual 2019 p_era values for comparison
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse_C = np.sqrt(mean_squared_error(y_test_actual, y_pred_C))
    rmse_list.append(rmse_C)

    # Calculate MAE
    mae_C = mean_absolute_error(y_test_actual, y_pred_C)
    mae_list.append(mae_C)
    
    # Calculate MAPE
    mape_C = mean_absolute_percentage_error(y_test_actual, y_pred_C)
    mape_list.append(mape_C)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_C}, MAE: {mae_C}, MAPE: {mape_C}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 38s 755ms/step - loss: 0.1496 - accuracy: 0.0025 - val_loss: 0.0457 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 24ms/step - loss: 0.0413 - accuracy: 0.0025 - val_loss: 0.0308 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0287 - accuracy: 0.0025 - val_loss: 0.0481 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0270 - accuracy: 0.0025 - val_loss: 0.0291 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0249 - accuracy: 0.0025 - val_loss: 0.0341 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0255 - accuracy: 0.0025 - val_loss: 0.0323 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0213 - accuracy: 0.0025 - val_loss: 0.0280 - val_accuracy: 0

In [6]:
# LSTM_D

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_D
    model_D = Sequential()
    model_D.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_D.add(LSTM(32, return_sequences=True))
    model_D.add(Dropout(rate=0.5))
    model_D.add(Flatten())
    model_D.add(Dense(512, activation="relu"))
    model_D.add(Dropout(rate=0.5))
    model_D.add(Dense(64, activation="relu"))
    model_D.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_D.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_D = model_D.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter data for the year 2019
    data_19 = common_data[common_data['year'] == 2019]

    # Scale the data for the year 2019
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Convert to time series data format (using data from 2016, 2017, 2018 to predict 2019)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict data for the year 2019
    y_pred_scaled_D = model_D.predict(X_2019_seq)

    # Inverse scale the predictions
    y_pred_D = scaler_y.inverse_transform(y_pred_scaled_D)

    # Restore the actual 2019 p_era values for comparison
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse_D = np.sqrt(mean_squared_error(y_test_actual, y_pred_D))
    rmse_list.append(rmse_D)

    # Calculate MAE
    mae_D = mean_absolute_error(y_test_actual, y_pred_D)
    mae_list.append(mae_D)
    
    # Calculate MAPE
    mape_D = mean_absolute_percentage_error(y_test_actual, y_pred_D)
    mape_list.append(mape_D)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_D}, MAE: {mae_D}, MAPE: {mape_D}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 22s 349ms/step - loss: 0.1115 - accuracy: 0.0025 - val_loss: 0.0303 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0450 - accuracy: 0.0000e+00 - val_loss: 0.0410 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0284 - accuracy: 0.0025 - val_loss: 0.0499 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0248 - accuracy: 0.0025 - val_loss: 0.0301 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0252 - accuracy: 0.0025 - val_loss: 0.0378 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0228 - accuracy: 0.0025 - val_loss: 0.0388 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0222 - accuracy: 0.0025 - val_loss: 0.0311 - val_accurac